In [1]:
import os
from ape import networks
import pandas as pd
import matplotlib.pyplot as plt
from CreditGuild.controllers.ActiveLendingTerms import fetch_terms_and_debt, fetch_user_stakes
from GeckoTerminal.models.API import fetch_prices, fetch_networks

## Fetch Stake

In [2]:
stake_minters = [
    {"surplusGuildMinter": "0xb94aaae7472a694dd959c8497b2f09730391dc52", "creditToken": "0xD5FD8456aa96aAA07c23605e9a8d2ce5f737F145"},
    {"surplusGuildMinter": "0x55ab4c8a5f11f8e62d7822d5aed778784df12afd", "creditToken": "0xaFBe44E79E9affB25CEd16D971933219d1d6EC8d"},
    {"surplusGuildMinter": "0x6995aa07b177918423d2127b885b67e7a3cec265", "creditToken": "0x8D8b654E2B6A0289D8f758cBCCc42aB387c67B61"},
    {"surplusGuildMinter": "0x71215ac6faf015aea177675543a8635beb08d183", "creditToken": "0x4139724fCB5E27d9F13Be9928E4436a32d12E00b"},
    {"surplusGuildMinter": "0xaa5bb0ffbfabec29a0df298b4b6a1a8e24cfe17e", "creditToken": "0x2170eea4D493e53e835d59f8aC559D007bC9A527"},
    {"surplusGuildMinter": "0x7c171a82f73e9c030cf9133606d883883c826acb", "creditToken": "0x808831eC837534E28A24097122875A6eE39C6A38"},
    {"surplusGuildMinter": "0x72c0a3d34aabd20db73a38c494f6e6be503f4a5b", "creditToken": "0x63Bebf6E5152ddeF1445e24B456f8128a9Fda7fc"},
    {"surplusGuildMinter": "0x0d81cf2515c02a7cdbd110c41e8dce2bb1983962", "creditToken": "0xb6CC9BbD136Bde153c380a987EF6b365bA9f7669"}
]

## Fetch Prices

In [ ]:
credit_token_map = [
    {"creditToken": "0x2170eea4D493e53e835d59f8aC559D007bC9A527", "address": "0x6a7661795c374c0bfc635934efaddff3a7ee23b6", "name": "DOLA"},
    {"creditToken": "0x4139724fCB5E27d9F13Be9928E4436a32d12E00b", "address": "0x221a0f68770658c15b525d0f89f5da2baab5f321", "name": "OD"},
    {"creditToken": "0x8D8b654E2B6A0289D8f758cBCCc42aB387c67B61", "address": "0x912ce59144191c1204e64559fe8253a0e49e6548", "name": "ARB"},
    {"creditToken": "0xD5FD8456aa96aAA07c23605e9a8d2ce5f737F145", "address": "0xaf88d065e77c8cc2239327c5edb3a432268e5831", "name": "USDC"},
    {"creditToken": "0xaFBe44E79E9affB25CEd16D971933219d1d6EC8d", "address": "0x82af49447d8a07e3bd95bd0d56f35241523fbab1", "name": "WETH"},
    {"creditToken": "0xb6CC9BbD136Bde153c380a987EF6b365bA9f7669", "address": "0x12275dcb9048680c4be40942ea4d92c74c63b844", "name": "eUSD"},
    {"creditToken": "0x808831eC837534E28A24097122875A6eE39C6A38", "address": "0x5979d7b546e38e414f7e9822514be443a4800529", "name": "wstETH"},
    {"creditToken": "0x63Bebf6E5152ddeF1445e24B456f8128a9Fda7fc", "address": "0x0022228a2cc5e7ef0274a7baa600d44da5ab5776", "name": "stUSD"},
    {"creditToken": "0xcFC43e1251bD7fd2f37766c083948497B7871AEb", "name": "test"},
    {"creditToken": "0xce805022e35fd808139d7E541F10D6e83420d84B", "name": "test"},
]

credit_underlying_prices = [
    {   "price": fetch_prices("arbitrum", token.get("address")),
        **token,
    } for token in credit_token_map if token.get("address")
]

## Fetch Data for Lending Terms and Debt

In [3]:
token_address = '0xb8ae64F191F829fC00A4E923D460a8F2E0ba3978'
terms_debt_csv_file_path = 'data/credit_guild_terms_debt.csv'

data = None
if os.path.exists(terms_debt_csv_file_path):
    data = pd.read_csv(terms_debt_csv_file_path)

with networks.parse_network_choice("arbitrum:mainnet:infura") as provider:
    data = data if not data is None else fetch_terms_and_debt(token_address)
    data.to_csv(terms_debt_csv_file_path, index=False, header=True)

In [ ]:
data['interestAmount'] = data['interestRate'] * data['totalIssuance']

In [ ]:
data.head()

In [ ]:
interest_summary = data.groupby('creditToken')['interestAmount'].sum().reset_index()

In [ ]:
merged_earned_interest = pd.merge(interest_summary, pd.DataFrame(credit_underlying_prices), on="creditToken", how="right")
merged_earned_interest['interestAmount'] = merged_earned_interest['interestAmount'].astype(float)
merged_earned_interest['interestDollarValue'] = merged_earned_interest['interestAmount'] * merged_earned_interest['price']

earned_interest = merged_earned_interest[["name", "interestDollarValue"]]

## Plots

### User Stakes

In [4]:
merged_staking = pd.merge(data, pd.DataFrame(stake_minters), on="creditToken", how="inner")

user_stakes = None
with networks.parse_network_choice("arbitrum:mainnet:infura") as provider:
    user_stakes = fetch_user_stakes("", merged_staking, debug=False)

In [10]:
user_stakes[user_stakes["creditToken"] == "0xaFBe44E79E9affB25CEd16D971933219d1d6EC8d"]

,termAddress,creditToken,surplusGuildMinter,userStake
89,0x7049e9D5E491CAe4d5215c9cc8E2Da3f3e172e1c,0xaFBe44E79E9affB25CEd16D971933219d1d6EC8d,0x55ab4c8a5f11f8e62d7822d5aed778784df12afd,0.000000
90,0x17224a9C3c4554ca8ACaA3B7ffC53baa9C8E8C41,0xaFBe44E79E9affB25CEd16D971933219d1d6EC8d,0x55ab4c8a5f11f8e62d7822d5aed778784df12afd,0.000000
91,0x0fa9e5815a34990c3Ca42703A576BE674d0f621A,0xaFBe44E79E9affB25CEd16D971933219d1d6EC8d,0x55ab4c8a5f11f8e62d7822d5aed778784df12afd,0.000000
92,0x9456E262a68bf2bC8Cb9265278D0f8AD657D8652,0xaFBe44E79E9affB25CEd16D971933219d1d6EC8d,0x55ab4c8a5f11f8e62d7822d5aed778784df12afd,0.000000
93,0x2173fe77E9ff8E75e72035D6F090711801C6B035,0xaFBe44E79E9affB25CEd16D971933219d1d6EC8d,0x55ab4c8a5f11f8e62d7822d5aed778784df12afd,0.000000
94,0x12E4cdAffD54b9Ce0555d7cc3DFCE151d50A1350,0xaFBe44E79E9affB25CEd16D971933219d1d6EC8d,0x55ab4c8a5f11f8e62d7822d5aed778784df12afd,0.000000
95,0xc382Ec61380b991D2029AAeDEA7547eC32A5B66B,0xaFBe44E79E9affB25CEd16D971933219d1d6EC8d,0x55ab4c8a5f11f8e62d7822d5aed778784df12afd,0.000000
96,0xaa19FcC1B1b06B9B82558bd1Da56462f85fc9b13,0xaFBe44E79E9affB25CEd16D971933219d1d6EC8d,0x55ab4c8a5f11f8e62d7822d5aed778784df12afd,0.000000
97,0x2E464418151424C0875610eE53573dF2fBb51d93,0xaFBe44E79E9affB25CEd16D971933219d1d6EC8d,0x55ab4c8a5f11f8e62d7822d5aed778784df12afd,0.000000
98,0x43eE18971eD50B0E57Ac2276ffaC5D4865f913BF,0xaFBe44E79E9affB25CEd16D971933219d1d6EC8d,0x55ab4c8a5f11f8e62d7822d5aed778784df12afd,0.000000


### Earned Interest

In [ ]:
colormap = plt.get_cmap('tab10') 

# Generate a list of colors from the colormap
colors = [colormap(i) for i in range(len(earned_interest))]

plt.bar(earned_interest['name'], earned_interest['interestDollarValue'], color=colors)
plt.xlabel('Token')
plt.ylabel('Earnings ($)')
plt.title('Projected Yearly Interest Earned')

# Show x labels clearly
plt.xticks(rotation=0)

# Display the plot
plt.show()
